# Version 01: Logistic Regression

In this first attempt, I'll use a simple logistic regression model to try to predict survival.

In [1]:
# Standard imports
import numpy as np
import pandas as pd

In [2]:
# Load the training data and take a look
train_data = pd.read_csv("data/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Load the test data and take a look
test_data = pd.read_csv("data/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
print("train_data shape: ", train_data.shape)
print("test_data shape: ", test_data.shape)

train_data shape:  (891, 12)
test_data shape:  (418, 11)


Now we have a quick preview of the datasets and can see that training set <code>train_data</code> contains 891 observations while the test set <code>test_data</code> contains 418 observations--and, notably, excludes the survival data. Our task will be to predict this.

Let's see whether we're dealing with any null values either set:

In [5]:
print("Train data")
print(train_data.isna().sum())
print()
print("Test data")
print(test_data.isna().sum())

Train data
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Test data
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


Age strikes me as a potentially predictive feature, but there are a lot of missing values here, so I'm not sure at the moment how useful it can be--particularly since we're missing age data from the test set, as well.

For this first effort, then, I'll focus on the features where we do have data: <code>Pclass</code> (ticket class), <code>Sex</code>, <code>SibSp</code> (# of siblings/spouses aboard), and <code>Parch</code> (# of parents/children aboard). Because <code>Pclass</code> and <code>Sex</code> are categorical, I'll one-hot encode those columns.

In [6]:
def prepare_data(train, test, features=['Pclass', 'Sex', 'SibSp', 'Parch']):

    X = pd.get_dummies(train[features], columns=['Pclass', 'Sex'])
    y = train['Survived']
    X_test = pd.get_dummies(test[features], columns=['Pclass', 'Sex'])
    
    return  X, y, X_test

In [7]:
X, y, X_test = prepare_data(train_data, test_data)
X

,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male
0,1,0,0,0,1,0,1
1,1,0,1,0,0,1,0
2,0,0,0,0,1,1,0
3,1,0,1,0,0,1,0
4,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...
886,0,0,0,1,0,0,1
887,0,0,1,0,0,1,0
888,1,2,0,0,1,1,0
889,0,0,1,0,0,0,1


There's our dataset. Let's see how a simple logistic regression model performs.

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

print("Average cross validation score: {:.4f}"
      .format(cross_val_score(model, X, y, cv=5).mean()))

Average cross validation score: 0.7991


Not bad for starters. We'll see if this can be improved in future notebooks. For now let's fit the model and make predictions from the test set.

In [9]:
model.fit(X, y)
y_test = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': y_test})
output

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [10]:
output.to_csv('submissions/submission1.csv', index=False)